<a href="https://colab.research.google.com/github/ayush572/Fake_News_Classification_LSTM/blob/main/Fake_News_Classification_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [5]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/c/fake-news/data#/")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ayushdg1
Your Kaggle Key: ··········


100%|██████████| 46.5M/46.5M [00:01<00:00, 28.4MB/s]



Extracting archive ./fake-news/fake-news.zip to ./fake-news


In [8]:
import pandas as pd
df = pd.read_csv('fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# dropping nan values
df = df.dropna()

In [11]:
# now removing the label as label is the output, so removing it in the X
X = df.drop('label', axis = 1)

In [12]:
# y contains only the output of the same
Y = df['label']

In [14]:
print(X.shape)
Y.shape

(18285, 4)


(18285,)

In [15]:
import tensorflow as tf

In [16]:
# importing those things that are necessary to use LSTM
from tensorflow.keras.layers import Embedding # to convert the one-hot encoded vectors to featurized vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences # helps to make the sentences of equal length
from tensorflow.keras.models import Sequential # to create a sequential model
from tensorflow.keras.preprocessing.text import one_hot # to convert the words to one_hot encoded vectors
from tensorflow.keras.layers import LSTM, Dense # using of LSTM and Dense layers
# Dense layer is being used as this is a classification problem, so that's why we require the same